# DATA TARGET 값

In [1]:
'''
1. 놀람 surprise
2. 두려움/공포 fear
3. 혐오 disgust
4. 행복 happiness
5. 슬픔 sadness
6. 분노 anger
7. 무표정/중립 neutral
'''

'\n1. 놀람 surprise\n2. 두려움/공포 fear\n3. 혐오 disgust\n4. 행복 happiness\n5. 슬픔 sadness\n6. 분노 anger\n7. 무표정/중립 neutral\n'

In [2]:
import kagglehub
path = kagglehub.dataset_download("shuvoalok/raf-db-dataset")
print("path:", path)


Using Colab cache for faster access to the 'raf-db-dataset' dataset.
path: /kaggle/input/raf-db-dataset


#DATASET 링크
[https://www.kaggle.com/datasets/samithsachidanandan/human-face-emotions/data](https://www.kaggle.com/datasets/samithsachidanandan/human-face-emotions/data)

In [3]:
import os, glob, zipfile
import kagglehub

# 1) kagglehub로 RAF-DB 다운로드 (zip or folder를 받을 수 있음)
path = kagglehub.dataset_download("shuvoalok/raf-db-dataset")
print("kagglehub path:", path)
print("list(path):", os.listdir(path)[:20])

# 2) path 안/주변에서 zip을 찾거나, 이미 DATASET이 풀려있으면 그대로 사용
def find_dataset_root(p):
    # 케이스 A: 이미 DATASET 폴더가 바로 있음
    if os.path.isdir(os.path.join(p, "DATASET")):
        return p

    # 케이스 B: p 자체가 DATASET일 수도 있음
    if os.path.basename(p) == "DATASET" and os.path.isdir(os.path.join(p, "train")):
        return os.path.dirname(p)

    # 케이스 C: zip이 있으면 /content/rafdb_extracted 로 풀기
    zips = glob.glob(os.path.join(p, "*.zip"))
    if not zips:
        # 상위에도 zip이 있을 수 있어 한 단계 더 탐색
        zips = glob.glob(os.path.join(os.path.dirname(p), "*.zip"))

    if zips:
        zpath = zips[0]
        out_dir = "/content/rafdb_extracted"
        os.makedirs(out_dir, exist_ok=True)
        with zipfile.ZipFile(zpath, "r") as z:
            z.extractall(out_dir)

        # extract 후 DATASET 찾기
        for root, dirs, files in os.walk(out_dir):
            if "DATASET" in dirs:
                return os.path.join(root)
    return None

dataset_root = find_dataset_root(path)
print("dataset_root:", dataset_root)

# 3) 최종 BASE_DIR 확정 (원래 코드가 기대하는 위치)
BASE_DIR = os.path.join(dataset_root, "DATASET")
TRAIN_ROOT = os.path.join(BASE_DIR, "train")
TEST_ROOT  = os.path.join(BASE_DIR, "test")

print("BASE_DIR:", BASE_DIR)
print("train exists?", os.path.exists(TRAIN_ROOT))
print("test exists?", os.path.exists(TEST_ROOT))
print("train subdirs:", os.listdir(TRAIN_ROOT)[:10])  # 1~7 나와야 정상


Using Colab cache for faster access to the 'raf-db-dataset' dataset.
kagglehub path: /kaggle/input/raf-db-dataset
list(path): ['test_labels.csv', 'DATASET', 'train_labels.csv']
dataset_root: /kaggle/input/raf-db-dataset
BASE_DIR: /kaggle/input/raf-db-dataset/DATASET
train exists? True
test exists? True
train subdirs: ['7', '2', '5', '3', '1', '4', '6']


In [4]:
!pip install torchinfo
!pip install torchmetrics

In [5]:
# Module import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torch.optim import Adam
import torchmetrics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import albumentations as A

from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from PIL import Image

In [6]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split


# BASE_DIR 예: /kaggle/input/raf-db-dataset/DATASET
TRAIN_ROOT = os.path.join(BASE_DIR, "train")
TEST_ROOT  = os.path.join(BASE_DIR, "test")

def build_df(root_dir):
    image_paths = []
    labels = []

    # root_dir 아래에 1~7 폴더가 있음
    for label_str in sorted(os.listdir(root_dir)):  # '1'~'7'
        class_dir = os.path.join(root_dir, label_str)
        if not os.path.isdir(class_dir):
            continue

        label = int(label_str)  # ✅ 폴더명이 라벨(1~7)

        for fname in os.listdir(class_dir):
            if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                image_paths.append(os.path.join(class_dir, fname))
                labels.append(label)

    return pd.DataFrame({"image_path": image_paths, "label": labels})

train_df_full = build_df(TRAIN_ROOT)
test_df       = build_df(TEST_ROOT)

train_df, val_df = train_test_split(
    train_df_full,
    test_size=0.2,
    stratify=train_df_full["label"],
    random_state=42
)

print("train/val/test:", train_df.shape, val_df.shape, test_df.shape)
print("train label counts:\n", train_df["label"].value_counts().sort_index())

# ✅ 경로 존재 확인(True 떠야 정상)
print("sample train path:", train_df.iloc[0]["image_path"])
print("exists?", os.path.exists(train_df.iloc[0]["image_path"]))

train/val/test: (9816, 2) (2455, 2) (3068, 2)
train label counts:
 label
1    1032
2     225
3     574
4    3817
5    1585
6     564
7    2019
Name: count, dtype: int64
sample train path: /kaggle/input/raf-db-dataset/DATASET/train/4/train_08289_aligned.jpg
exists? True


# 채환
데이터 가공 및 Dataset & DataLoader 만들기

In [7]:
import cv2
from torch.utils.data import Dataset, DataLoader

# CNN Dataset
class Custom_Dataset(Dataset):
    # 여기서 transform은 albumentations transform이라고 가정.
    # 모든 image는 OpenCV로 다룸.
    # 따라서 마지막에 last channel -> first channel로 바꿔주는 로직이 별도로 필요.
    def __init__(self, image_paths, targets=None, transform=None):
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform

    # 전체 건수 반환
    def __len__(self):
        return len(self.image_paths)

    # 주요 메커니즘
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        # image는 ndarry.
        image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

        # albumentation 별도 적용.
        # albumentation에서 ToTensorV2를 별도로 적용.
        # 사실 이 로직이 작동하지 않으면 오류 발생.
        if self.transform is not None:
            image = self.transform(image=image)['image']

        if self.targets is not None:
            target = torch.tensor(self.targets[idx])
            return image, target
        else:
            return image

In [8]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    #A.VerticalFlip(p=0.5),
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2(),
])

test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2(),
])

In [9]:
train_dataset = Custom_Dataset(train_df['image_path'].values, train_df['label'].values, transform=train_transform)
val_dataset = Custom_Dataset(val_df['image_path'].values, val_df['label'].values, transform=val_transform)
test_dataset = Custom_Dataset(test_df['image_path'].values, test_df['label'].values, transform=test_transform)

In [10]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, pin_memory=True)

In [11]:
model = None # 이 줄은 이전에 SyntaxError를 발생시켰습니다. 문법 오류를 해결하기 위해 None으로 초기화합니다.

In [12]:
def create_pretrained_model(model_name='alexnet', classifier_layer=None, image_size=[224, 224], make_summary=False):
    if model_name == 'alexnet':
        model = models.alexnet(weights='DEFAULT')
        model.classifier = classifier_layer
    elif model_name == 'resnet50':
        model = models.resnet50(weights='DEFAULT')
        model.fc = classifier_layer
    elif model_name == 'resnet101':
        model = models.resnet101(weights='DEFAULT')
        model.fc = classifier_layer
    elif model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights='DEFAULT')
        model.classifier = classifier_layer
    elif model_name == 'efficientnet_b4':
        model = models.efficientnet_b4(weights='DEFAULT')
        model.classifier = classifier_layer
    elif model_name == 'efficientnet_v2_s':
        model = models.efficientnet_v2_s(weights='DEFAULT')
        model.classifier = classifier_layer
    elif model_name == 'efficientnet_v2_m':
        model = models.efficientnet_v2_m(weights='DEFAULT')
        model.classifier = classifier_layer
    elif model_name == 'efficientnet_v2_l':
        model = models.efficientnet_v2_l(weights='DEFAULT')
        model.classifier = classifier_layer
    elif model_name == 'convnext_base':
        model = models.convnext_base(weights='DEFAULT')
        model.classifier[2] = classifier_layer
    elif model_name == 'convnext_small':
        model = models.convnext_small(weights='DEFAULT')
        model.classifier[2] = classifier_layer
    else:
        print("🧨🧨 [ERROR] 모델 이름이 존재하지 않습니다. 🧨🧨")
        return None

    if make_summary:
        # 모델 정보 요약(model summary)
        print(torchinfo.summary(model, input_size=[1, 3] + image_size,
                  col_names=['output_size', 'num_params', 'trainable'],
                  row_settings=['depth', 'var_names'],
                  depth=3))

    return model

classifier_layer = nn.Sequential(
    nn.Linear(in_features=1280, out_features=2),
)

eff_model = create_pretrained_model(
    model_name = 'efficientnet_b0',
    classifier_layer = classifier_layer,
    make_summary=True
    )

Layer (type (var_name):depth-idx)                                 Output Shape              Param #                   Trainable
EfficientNet (EfficientNet)                                       [1, 2]                    --                        True
├─Sequential (features): 1-1                                      [1, 1280, 7, 7]           --                        True
│    └─Conv2dNormActivation (0): 2-1                              [1, 32, 112, 112]         --                        True
│    │    └─Conv2d (0): 3-1                                       [1, 32, 112, 112]         864                       True
│    │    └─BatchNorm2d (1): 3-2                                  [1, 32, 112, 112]         64                        True
│    │    └─SiLU (2): 3-3                                         [1, 32, 112, 112]         --                        --
│    └─Sequential (1): 2-2                                        [1, 16, 112, 112]         --                        True
│    │    └─M

# 서현
모델 학습 로직 만들기

In [13]:
# CNN 학습 Class
class Trainer:
  def __init__(self, model=None, train_dataloader=None, val_dataloader=None, loss_fn=None, auc_metric=None, acc_metric=None, optimizer=None, scheduler=None):
    self.model = model
    self.val_dataloader = val_dataloader
    self.loss_fn = loss_fn    # 손실 함수
    self.auc_metric = auc_metric    # 평가 지표: AUC
    self.acc_metric = acc_metric    # 평가 지표: 정확도 (Accuracy)
    self.optimizer = optimizer    # 최적화 도구
    self.scheduler = scheduler    # 학습률 스케줄러

    # 가장 좋은 성능을 기록하기 위한 변수 초기화
    self.best_val_f1 = 0.0    # 최고 점수를 저장할 변수
    self.best_model_state = None    # 최고 점수일 때의 모델 가중치를 저장할 변수

  # 1번 에포크 학습 함수
  def train_epoch(self, train_dataloader):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # 모델과 평가 지표들을 해당 장치(GPU/CPU)로 이동
    self.model = self.model.to(device)
    if self.acc_metric is not None:
      self.acc_metric = self.acc_metric.to(device)
    if self.auc_metric is not None:
      self.auc_metric = self.auc_metric.to(device)

    # 에포크 동안의 평균 손실(Loss)을 계산하기 위한 변수 초기화
    avg_loss = 0
    sum_loss = 0

    # 모델 학습 설정
    self.model.train()

    # tqdm을 사용하여 진행률 바(progress bar) 생성
    with tqdm(total = len(train_dataloader), desc="[Training...]", leave=True) as progress_bar:
      for batch_idx, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)

        # 라벨 보정
        labels = labels - 1


        # 순전파 (Forward Pass): 모델이 이미지를 보고 예측값(logits)을 내놓음
        logits = self.model(images)

        # 손실 계산: 예측값(logits)과 정답(labels)의 차이(Loss)를 계산
        loss = self.loss_fn(logits, labels)

        # 손실 기록: 현재 배치의 손실을 누적하고 평균을 계산
        sum_loss += loss.item()
        avg_loss = sum_loss / (batch_idx + 1)

        # 평가 지표 업데이트
        if self.auc_metric is not None:
          self.auc_metric.update(F.softmax(logits, dim=-1), labels)
        if self.acc_metric is not None:
          self.acc_metric.update(F.softmax(logits, dim=-1).argmax(dim=-1), labels)

        # 역전파
        self.optimizer.zero_grad()    # 이전 배치의 기울기(gradient) 초기화
        loss.backward()    # 현재 오차에 대한 기울기 계산 (역전파)
        self.optimizer.step()    # 계산된 기울기를 이용해 모델 파라미터 업데이트

        # 진행률 바 1칸 증가
        progress_bar.update(1)

        # 20번째 배치마다 또는 마지막 배치일 때 현재 상태 출력
        if batch_idx % 20 == 0 or batch_idx+1 == len(train_dataloader):
          current_auc = self.auc_metric.compute().item() if self.auc_metric is not None else 0.0
          current_acc = self.acc_metric.compute().item() if self.acc_metric is not None else 0.0

          progress_bar.set_postfix({
            "Train_Loss": avg_loss,
            "Train_ACC": current_acc,
            "Train_AUC": current_auc,
          })

      if self.auc_metric is not None:
        self.auc_metric.reset()
      if self.acc_metric is not None:
        self.acc_metric.reset()

    return avg_loss, current_acc, current_auc

  # 1번 에포크 검증 함수
  def validate_epoch(self, val_dataloader):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    self.model = self.model.to(device)
    if self.acc_metric is not None:
      self.acc_metric = self.acc_metric.to(device)
    if self.auc_metric is not None:
      self.auc_metric = self.auc_metric.to(device)

    # 평균 loss를 구하기 위한 변수 초기화
    avg_loss = 0
    sum_loss = 0

    # 모델 평가로 설정
    self.model.eval()

    # 기울기 계산 비활성화
    with torch.no_grad():
      with tqdm(total = len(val_dataloader), desc="[Validating..]", leave=True) as progress_bar:
        for batch_idx, (images, labels) in enumerate(val_dataloader):
          images = images.to(device)
          labels = labels.to(device)

          # 라벨 보정
          labels = labels - 1

          logits = self.model(images)
          loss = self.loss_fn(logits, labels)
          sum_loss += loss.item()
          avg_loss = sum_loss / (batch_idx + 1)

          if self.auc_metric is not None:
            self.auc_metric.update(F.softmax(logits, dim=-1), labels)
          if self.acc_metric is not None:
            self.acc_metric.update(F.softmax(logits, dim=-1).argmax(dim=-1), labels)

          # 진행률 바 업데이트
          progress_bar.update(1)

          if batch_idx % 20 == 0 or batch_idx + 1 == len(val_dataloader):
            current_acc = self.acc_metric.compute().item() if self.acc_metric is not None else 0.0
            current_auc = self.auc_metric.compute().item() if self.auc_metric is not None else 0.0
            progress_bar.set_postfix({
              "Validate_Loss": avg_loss,
              "Validate_ACC": current_acc,
              "Validate_AUC": current_auc,
            })

        if self.acc_metric is not None:
          self.acc_metric.reset()
        if self.auc_metric is not None:
          self.auc_metric.reset()

    return avg_loss, current_acc, current_auc


  # fit 함수: 전체 함수 루프 관리
  def fit(self, epochs, train_dataloader=None, val_dataloader=None):
    # 학습 기록을 저장할 딕셔너리 생성
    history = {
      'train_loss' : [], 'val_loss' : [],
      'train_acc' : [], 'val_acc' : [],
      'train_auc' : [], 'val_auc' : [],
    }

    print("모델 학습 시작!")
    print("=" * 50)

    for epoch in range(1, epochs+1):
      print(f"\nEpoch {epoch}/{epochs}")
      print("-" * 30)

      # 학습 실행
      if train_dataloader is not None:
        train_loss, train_acc, train_auc = self.train_epoch(train_dataloader)
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['train_auc'].append(train_auc)
      else:
        print("학습 데이터로더가 존재하지 않음!")
        return history

      # 검증 실행
      if val_dataloader is not None:
        val_loss, val_acc, val_auc = self.validate_epoch(val_dataloader)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_auc'].append(val_auc)

        # 학습률 스케줄러 업데이트
        if self.scheduler is not None:
          self.scheduler.step(val_loss)

        # 최고 모델 저장
        if val_acc > self.best_val_f1:
          self.best_val_f1 = val_acc
          self.best_model_state = self.model.state_dict().copy()
          print(f"새로운 최고 ACC 점수: {self.best_val_f1:.4f}")

      # 결과 출력
      print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Train AUC: {train_auc:.4f}")
      if val_dataloader is not None:
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | Val AUC: {val_auc:.4f}")
        if self.scheduler is not None:
          print(f"Learning Rate: {self.optimizer.param_groups[0]['lr']:.6f}")

        print("=" * 50)

    # 최고 모델 복원
    if self.best_model_state is not None:
      self.model.load_state_dict(self.best_model_state)
      print(f"최고 ACC 점수 모델로 복원 완료: {self.best_val_f1:.4f}")

    return history


  # 모델 평가
  def evaluate(self, eval_dataloader):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    self.model = self.model.to(device)
    if self.acc_metric is not None:
      self.acc_metric = self.acc_metric.to(device)
    if self.auc_metric is not None:
      self.auc_metric = self.auc_metric.to(device)

    # 평균 loss를 구하기 위한 변수 초기화
    avg_loss = 0
    sum_loss = 0

    # 모델 평가로 설정
    self.model.eval()

    with torch.no_grad():
      with tqdm(total = len(eval_dataloader), desc="[Evaluating..]", leave=True) as progress_bar:
        for batch_idx, (images, labels) in enumerate(eval_dataloader):
          images = images.to(device)
          labels = labels.to(device)

          # 라벨 보정
          labels = labels -1

          logits = self.model(images)
          loss = self.loss_fn(logits, labels)
          sum_loss += loss.item()
          avg_loss = sum_loss / (batch_idx+1)

          if self.auc_metric is not None:
            self.auc_metric.update(F.softmax(logits, dim=-1), labels)
          if self.acc_metric is not None:
            self.acc_metric.update(F.softmax(logits, dim=-1).argmax(dim=-1), labels)

          # 진행률 바 업데이트
          progress_bar.update(1)

          if batch_idx % 20 == 0 or batch_idx + 1 == len(eval_dataloader):
            current_acc = self.acc_metric.compute().item() if self.acc_metric is not None else 0.0
            current_auc = self.auc_metric.compute().item() if self.auc_metric is not None else 0.0
            progress_bar.set_postfix({
              "Evaluate_Loss": avg_loss,
              "Evaluate_ACC": current_acc,
              "Evaluate_AUC": current_auc,
            })

        if self.acc_metric is not None:
          self.acc_metric.reset()
        if self.auc_metric is not None:
          self.auc_metric.reset()

    return avg_loss, current_acc, current_auc

  # 마지막으로 학습된 모델 반환
  def get_trained_model(self):
    return self.model

In [14]:

# 학습을 위한 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")


# 7개 클래스 분류를 위한 분류기 레이어
classifier_layer = nn.Sequential(
  nn.Dropout(0.5),
  nn.Linear(in_features=1280, out_features=512),
  nn.ReLU(),
  nn.Dropout(0.3),
  nn.Linear(in_features=512, out_features=7)
)

# 모델 생성
model = create_pretrained_model(
  model_name = 'efficientnet_b0',
  classifier_layer = classifier_layer,
  make_summary=True
)

model = model.to(device)

# 학습 설정
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

# 평가 지표 설정
auc_metric = torchmetrics.AUROC(task="multiclass", num_classes=7)
acc_metric = torchmetrics.Accuracy(task="multiclass", num_classes=7)

# Trainer 클래스 인스턴스 생성 및 데이터 주입
trainer = Trainer(
  model=model,
  train_dataloader=train_dataloader,
  val_dataloader=val_dataloader,
  loss_fn=loss_fn,
  auc_metric=auc_metric,
  acc_metric=acc_metric,
  optimizer=optimizer,
  scheduler=scheduler
)

사용 디바이스: cpu
Layer (type (var_name):depth-idx)                                 Output Shape              Param #                   Trainable
EfficientNet (EfficientNet)                                       [1, 7]                    --                        True
├─Sequential (features): 1-1                                      [1, 1280, 7, 7]           --                        True
│    └─Conv2dNormActivation (0): 2-1                              [1, 32, 112, 112]         --                        True
│    │    └─Conv2d (0): 3-1                                       [1, 32, 112, 112]         864                       True
│    │    └─BatchNorm2d (1): 3-2                                  [1, 32, 112, 112]         64                        True
│    │    └─SiLU (2): 3-3                                         [1, 32, 112, 112]         --                        --
│    └─Sequential (1): 2-2                                        [1, 16, 112, 112]         --                        True


In [ ]:

# 학습 실행
history = trainer.fit(epochs=15, train_dataloader=train_dataloader, val_dataloader=val_dataloader)


모델 학습 시작!

Epoch 1/15
------------------------------


[Training...]:   0%|          | 0/614 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
[Training...]:   0%|          | 1/614 [00:05<57:11,  5.60s/it]/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
[Training...]:   1%|▏         | 9/614 [00:39<34:39,  3.44s/it, Train_Loss=1.9, Train_ACC=0.312, Train_AUC=0.534]

In [ ]:

# 학습 결과 시각화
def plot_training_history(history):
  fig, axes = plt.subplots(1,3,figsize=(15,5))

  epochs = range(1, len(history['train_loss']) + 1)

  # Loss 그래프
  axes[0].plot(epochs, history['train_loss'], 'b-', label='Train Loss')
  axes[0].plot(epochs, history['val_loss'], 'r-', label='Val Loss')
  axes[0].set_title('Training and Validation Loss')
  axes[0].set_xlabel('Epochs')
  axes[0].set_ylabel('Loss')
  axes[0].legend()
  axes[0].grid(True)

  # Accuracy 그래프
  axes[1].plot(epochs, history['train_acc'], 'b-', label='Train Acc')
  axes[1].plot(epochs, history['val_acc'], 'r-', label='Val Acc')
  axes[1].set_title('Training and Validation Accuracy')
  axes[1].set_xlabel('Epochs')
  axes[1].set_ylabel('Accuracy')
  axes[1].legend()
  axes[1].grid(True)

  # AUC 그래프
  axes[2].plot(epochs, history['train_auc'], 'b-', label='Train AUC')
  axes[2].plot(epochs, history['val_auc'], 'r-', label='Val AUC')
  axes[2].set_title('Training and Validation AUC')
  axes[2].set_xlabel('Epochs')
  axes[2].set_ylabel('AUC')
  axes[2].legend()
  axes[2].grid(True)

  plt.tight_layout()
  plt.show()

# 학습 결과 시각화 실행
plot_training_history(history)

# 최종 모델 저장
torch.save(trainer.get_trained_model().state_dict(), 'emotion_classification_model.pth')
print("모델 저장 완료: emotion_classification_model.pth")

# 명수
모델 예측 로직 만들기

In [ ]:
# --- Predictor 클래스 정의 ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt
import numpy as np

class EmotionPredictor:
    def __init__(self, model_path, device=None):
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # 감정 레이블
        self.emotions = ["Surprise", "Fear", "Disgust", "Happiness", "Sadness", "Anger", "Neutral"]

        # 모델 구조 생성
        classifier_layer = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features=1280, out_features=512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(in_features=512, out_features=7)
        )
        self.model = create_pretrained_model(model_name='efficientnet_b0', classifier_layer=classifier_layer)

        # 가중치 로드
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.to(self.device)
        self.model.eval()
        self.transform = test_transform # 전처리 활용

    def predict(self, image_path):
        image_bgr = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

        transformed = self.transform(image=image_rgb)['image']
        input_tensor = transformed.unsqueeze(0).to(self.device)

        with torch.no_grad():
            logits = self.model(input_tensor)
            probs = F.softmax(logits, dim=-1).cpu().numpy()[0] # 모든 감정의 확률값
            pred_idx = np.argmax(probs)

        return image_rgb, probs, pred_idx

In [ ]:
# --- 분석 결과 시각화 함수 정의 ---
def visualize_emotion_analysis(image_path, predictor):
    image_rgb, probs, pred_idx = predictor.predict(image_path)

    # 시각화 설정 (1행 2열)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # 1. 왼쪽: 이미지 출력
    ax1.imshow(image_rgb)
    ax1.set_title(f"Predicted: {predictor.emotions[pred_idx]} ({probs[pred_idx]*100:.2f}%)",
                 fontsize=15, fontweight='bold', pad=20)
    ax1.axis('off')

    # 2. 오른쪽: 감정별 확률 막대 그래프
    colors = ['gray'] * 7
    colors[pred_idx] = 'royalblue' # 가장 높은 확률의 막대 색상 변경

    y_pos = np.arange(len(predictor.emotions))
    ax2.barh(y_pos, probs * 100, align='center', color=colors)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(predictor.emotions, fontsize=12)
    ax2.invert_yaxis()  # 상단부터 보이게 정렬
    ax2.set_xlabel('Probability (%)', fontsize=12)
    ax2.set_title('Emotion Probability Distribution', fontsize=15, fontweight='bold', pad=20)
    ax2.set_xlim(0, 100)

    # 확률 수치 표시
    for i, v in enumerate(probs):
        ax2.text(v * 100 + 1, i, f"{v*100:.1f}%", color='black', va='center', fontweight='bold')

    plt.tight_layout()
    plt.show()

In [ ]:
import torch
from torch.utils.data import DataLoader

# test_df에 label이 있는 경우 (현재 폴더 구조상 test/1~7이면 label 존재)
test_dataset = Custom_Dataset(
    test_df["image_path"].values,
    test_df["label"].values,
    transform=test_transform  # 보통 val/test transform
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [ ]:
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

idx_to_emotion = {
    0: "surprise",
    1: "fear",
    2: "disgust",
    3: "happiness",
    4: "sadness",
    5: "anger",
    6: "neutral"
}
class_names = [idx_to_emotion[i] for i in range(7)]

# 1) 예측/정답 수집
y_true, y_pred = [], []

with torch.no_grad():
    for x, y in test_dataloader:
        x = x.to(device)
        logits = model(x)
        preds = torch.argmax(logits, dim=1).cpu().numpy()

        # ✅ (중요) 라벨이 1~7이면 0~6으로 변환 (학습에서 labels-1 했던 경우)
        y_np = (y.numpy() - 1)

        y_true.extend(y_np.tolist())
        y_pred.extend(preds.tolist())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# 2) 전체 정확도
acc = accuracy_score(y_true, y_pred)
print(f"\n✅ Test Accuracy: {acc*100:.2f}%  ({acc:.4f})\n")

# 3) 분류 리포트를 '표'로 보기 좋게
report_dict = classification_report(
    y_true, y_pred,
    target_names=class_names,
    digits=4,
    output_dict=True,
    zero_division=0
)

# 클래스 행만 뽑아서 표로 정리
rows = []
for name in class_names:
    rows.append([name,
                 report_dict[name]["precision"],
                 report_dict[name]["recall"],
                 report_dict[name]["f1-score"],
                 int(report_dict[name]["support"])])

report_df = pd.DataFrame(rows, columns=["class", "precision", "recall", "f1", "support"])
report_df_sorted = report_df.sort_values("f1", ascending=False).reset_index(drop=True)

print("📌 Class-wise metrics (sorted by F1):")
display(report_df_sorted)

# 4) 예측 분포 / 정답 분포 (편향 직관 확인)
true_counts = np.bincount(y_true, minlength=7)
pred_counts = np.bincount(y_pred, minlength=7)

dist_df = pd.DataFrame({
    "class": class_names,
    "true_count": true_counts,
    "pred_count": pred_counts
})
print("\n📌 Class distribution (true vs pred):")
display(dist_df)

plt.figure(figsize=(10,4))
plt.bar(dist_df["class"], dist_df["true_count"])
plt.title("True label distribution (Test)")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.bar(dist_df["class"], dist_df["pred_count"])
plt.title("Predicted label distribution (Model output)")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.show()

# 5) 혼동행렬을 '표'로 + 히트맵(수치/비율 둘 다)
cm = confusion_matrix(y_true, y_pred, labels=np.arange(7))
cm_df = pd.DataFrame(cm, index=[f"true_{c}" for c in class_names],
                        columns=[f"pred_{c}" for c in class_names])

print("\n📌 Confusion Matrix (counts):")
display(cm_df)

# 비율(행 기준) = 각 true 클래스에서 어디로 갔는지 %
cm_row_sum = cm.sum(axis=1, keepdims=True)
cm_norm = np.divide(cm, cm_row_sum, out=np.zeros_like(cm, dtype=float), where=cm_row_sum!=0)
cm_norm_df = pd.DataFrame(cm_norm, index=[f"true_{c}" for c in class_names],
                             columns=[f"pred_{c}" for c in class_names])

print("\n📌 Confusion Matrix (row-normalized, %):")
display((cm_norm_df*100).round(1))

plt.figure(figsize=(8,6))
plt.imshow(cm, aspect="auto")
plt.title("Confusion Matrix (Counts)")
plt.xticks(np.arange(7), class_names, rotation=30, ha="right")
plt.yticks(np.arange(7), class_names)
plt.colorbar()
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,6))
plt.imshow(cm_norm, aspect="auto")
plt.title("Confusion Matrix (Row-normalized)")
plt.xticks(np.arange(7), class_names, rotation=30, ha="right")
plt.yticks(np.arange(7), class_names)
plt.colorbar()
plt.tight_layout()
plt.show()

# 6) 가장 많이 헷갈린 TOP 오분류 쌍 출력 (직관적으로 “어디가 문제인지”)
pairs = []
for i in range(7):
    for j in range(7):
        if i == j:
            continue
        pairs.append((cm[i, j], class_names[i], class_names[j]))

pairs.sort(reverse=True, key=lambda x: x[0])

print("\n🔥 Top-10 confusions (true -> pred):")
for k in range(10):
    cnt, t, p = pairs[k]
    print(f"{k+1:2d}) {t:10s} -> {p:10s} : {cnt}")


In [ ]:
# --- 실제 테스트 실행 ---
from google.colab import files

# 1. 모델 객체 생성
predictor = EmotionPredictor(model_path='emotion_classification_model.pth')

# 2. 테스트용 이미지 업로드
print("분석할 사진을 업로드해 주세요 (jpg, png 등)")
uploaded = files.upload()

# 3. 모든 업로드 파일에 대해 시각화 실행
for filename in uploaded.keys():
    print(f"\n[ {filename} 분석 결과 ]")
    visualize_emotion_analysis(filename, predictor)